In [19]:
import torch
import torchvision.datasets as datasets
from tqdm import tqdm
from torch import nn
from ipynb.fs.full.model import VariationalAutoEncoder
from torchvision import transforms
from torchvision.utils import save_image  
from torch.utils.data import DataLoader, RandomSampler
import numpy as np
import matplotlib.pyplot as plt
import os
from mpl_toolkits import mplot3d
import cv2
from moviepy.editor import ImageSequenceClip

### Importing the model

In [2]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
INPUT_DIM = 256
INIT_DIM = 8
LATENT_DIM = 3
NUM_EPOCHS = 50
BATCH_SIZE = 1
LR_RATE = 3e-4
KERNEL_SIZE = 4

In [3]:
# Dataset Loading
data_path = 'dataset' # setting path
transform = transforms.Compose([transforms.Resize((INPUT_DIM, INPUT_DIM)),   # sequence of transformations to be done
                                transforms.Grayscale(num_output_channels=1), # on each image (resize, greyscale,
                                transforms.ToTensor()])                      # convert to tensor)

dataset = datasets.ImageFolder(root=data_path, transform=transform) # read data from folder

train_loader = DataLoader(dataset=dataset, batch_size=BATCH_SIZE, shuffle=True) # create dataloader object

model = VariationalAutoEncoder(init_dim=INIT_DIM, latent_dim=LATENT_DIM, kernel_size=KERNEL_SIZE).to(DEVICE) # initializing model object

optimizer = torch.optim.Adam(model.parameters(), lr=LR_RATE) # defining optimizer
loss_fn = nn.BCELoss(reduction='sum') # define loss function

In [5]:
model = VariationalAutoEncoder(init_dim=INIT_DIM, latent_dim=LATENT_DIM, kernel_size=KERNEL_SIZE)
model.load_state_dict(torch.load('models/model_256x'))

<All keys matched successfully>

### Generating stack from image A to image B

In [6]:
def get_random_sample(mu, sigma):
    epsilon = torch.randn_like(sigma)
    sample = mu + sigma*epsilon
    return sample

In [14]:
def reconstruct(z): # img is a tensor
    out = model.decode(z)
    out = out.view(-1, 1, 256, 256)
    save_image(out, 'temp.png')
    out = cv2.imread('temp.png')
    os.remove('temp.png')
    return out

In [130]:
def generate_stack(img_A,img_B,n_steps=20):
    
    with torch.no_grad():
        mu_A, sigma_A = model.encode(img_A.view(1,256,256)) # encoding of img A
        mu_B, sigma_B = model.encode(img_B.view(1,256,256)) # encoding of img B
        
    
    z_A = get_random_sample(mu_A, sigma_A)
    z_B = get_random_sample(mu_B, sigma_B)

    reconstructed_A = reconstruct(z_A)
    reconstructed_B = reconstruct(z_B)

    alpha = np.linspace(0, 1, n_steps)
    zs = [(1-a)*z_A + a*z_B for a in alpha]
    
    frames = []
    for i,z in enumerate(zs):
        img = reconstruct(z)
        thresh = 127
        im_bw = cv2.threshold(img, thresh, 255, cv2.THRESH_BINARY)[1]
        plt.imsave(f'output/interpolation/{i}.png', im_bw)
        
        # labeling the frame for the gif
        text = f'{i}'
        pos = (0,250)
        font = cv2.FONT_HERSHEY_SIMPLEX
        frame = cv2.putText(im_bw, text, org=pos, color=(255,0,0), fontFace = font, fontScale = .5)
        frames.append(frame)
    frames = np.array(frames)
        
    clip = ImageSequenceClip(list(frames), fps=60)
    clip.write_gif(f'output/interpolation/interpolation_{n_steps}.gif', fps=60)
    return frames, zs

In [131]:
frames, zs = generate_stack(dataset[0][0], dataset[200][0], 200)

MoviePy - Building file output/interpolation/interpolation_200.gif with imageio.


In [134]:
paths = sorted(os.listdir('dataset/0'))[:200]
frames = []
for i,path in enumerate(paths):
    img = cv2.imread(f'dataset/0/{path}')
    img = cv2.resize(img,(256,256))
    thresh = 127
    im_bw = cv2.threshold(img, thresh, 255, cv2.THRESH_BINARY)[1]
    plt.imsave(f'output/interpolation/realdata/{i}.png', im_bw)
    
    # labeling the frame for the gif
    text = f'{i}'
    pos = (0,250)
    font = cv2.FONT_HERSHEY_SIMPLEX
    frame = cv2.putText(im_bw, text, org=pos, color=(255,0,0), fontFace = font, fontScale = .5)
    frames.append(frame)
    
clip = ImageSequenceClip(frames, fps=60)
clip.write_gif(f'output/interpolation/realdata_256.gif', fps=60)

MoviePy - Building file output/interpolation/realdata_256.gif with imageio.
